Detección de caras con webcam

In [4]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

En el bucle de procesamiento, las teclas 'd' y 'e' `permiten respectivamenet cambiar de modelo de detección de caras, y en su caso de máscara de detección del rostro.

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [21]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 0
imodoE = 0

debug = 0

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    for (x, y) in shape:
                        cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)

                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)


                # Normalize and show
                # color channels
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))
                cv2.imshow("Normalized", NormBGR)


    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))

    # Show resulting image
    cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    if imodoF == 1 or imodoF == 2:
        cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    # Face detector change
    elif tec & 0xFF == ord('d'):
        imodoF = imodoF + 1
        if imodoF >= len(FDet.FaceDetectors):
            imodoF = 0
    #Eye detector change
    elif tec & 0xFF == ord('e'):
        imodoE = imodoE + 1
        if imodoE >= len(FDet.EyeDetectors):
            imodoE = 0


Camera 0


## TAREA

In [27]:
import cv2
import FaceNormalizationUtils as faceutils
import FaceDetectors

class PiggyFilter:
    def __init__(self, earl_str, earr_str, nose_str):
        self.earl_str = earl_str
        self.earr_str = earr_str
        self.nose_str = nose_str
    
    def _set_images(self, mode):
        earl_img = cv2.imread(self.earl_str, mode)
        earr_img = cv2.imread(self.earr_str, mode)
        nose_img = cv2.imread(self.nose_str, mode)

        self.earl_img = cv2.resize(earl_img, self.dim_ears)
        self.earr_img = cv2.resize(earr_img, self.dim_ears)
        self.nose_img = cv2.resize(nose_img, self.dim_nose)

    def _set_images_pos(self, shape):
        self.earl_pos = (shape[17][0]-int(self.dim_ears[0]/2), shape[17][1]-int(self.dim_ears[1]))
        self.earr_pos = (shape[26][0]-int(self.dim_ears[0]/2), shape[26][1]-int(self.dim_ears[1]))
        self.nose_pos = (shape[33][0]-int(self.dim_nose[0]/2), shape[33][1]-int(self.dim_nose[1]))

    def _set_dims(self, width):
        self.dim_ears = (int(width/2), int(width/2))
        self.dim_nose = (int(width/2), int(width/4))

    def _combine_images(self, frame, image, image_pos, dim):
        frame_roi = frame[image_pos[1]:image_pos[1]+dim[1], image_pos[0]:image_pos[0]+dim[0]]
        image_roi = image[:, :, 0:3]

        mask = image[:, :, 3]
        inversed_mask = cv2.bitwise_not(mask)

        frame_masked = cv2.bitwise_and(frame_roi, frame_roi, mask=inversed_mask)
        image_masked = cv2.bitwise_and(image_roi, image_roi, mask=mask)

        fused = cv2.add(frame_masked, image_masked)
        frame[image_pos[1]:image_pos[1]+dim[1], image_pos[0]:image_pos[0]+dim[0]] = fused
        
        return frame
    
    def apply(self, frame, values):
        face, eyes, shape = values
        [face_x, _, face_width, _] = face

        if face_x <= -1:
           return frame
         
        [leye_x, _, _, _] = eyes

        if leye_x <= -1:
            return frame

        self._set_dims(width = face_width)
        self._set_images(mode = cv2.IMREAD_UNCHANGED)
        self._set_images_pos(shape = shape)

        frame = self._combine_images(frame, self.earl_img, self.earl_pos, self.dim_ears)
        frame = self._combine_images(frame, self.earr_img, self.earr_pos, self.dim_ears)
        frame = self._combine_images(frame, self.nose_img, self.nose_pos, self.dim_nose)

        return frame

def capture_cam():
    vid = cv2.VideoCapture(0)
    cam_text = "Camera 0"

    if not vid.isOpened():
        vid = cv2.VideoCapture(1)
        if not vid.isOpened():
            vid = cv2.VideoCapture(0)
            if not vid.isOpened():
                print('Camera error')
                exit(0)
        else:
            cam_text = "Camera 1"

    print(cam_text)

    vid.set(3,640);
    vid.set(4,480);

    return vid

vid = capture_cam()
FDet = FaceDetectors.FaceDetector()
piggyFilter = PiggyFilter('./images/left_ear.png', './images/right_ear.png', './images/nose.png')

while True:
    ret, frame = vid.read()
    # for HS normalization
    B, G, R = cv2.split(frame)

    # search face
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
    if values is not None:
        frame = piggyFilter.apply(frame = frame, values = values)        

    cv2.imshow('Cam', frame)

    # stop with esc key
    if cv2.waitKey(20) == 27:
        break

# close windows and release camera
vid.release()
cv2.destroyAllWindows()

Camera 0
